# Tx_Tx

The main focus of this notebook is to demonstrate how to create a Tx_Tx catalog but can also be used to generally create one single catalog from two seperate catalogs.

In [ ]:
import numpy as np
from astropy.table import Table, join

Read in the files as Astropy Tables.

In [ ]:
# If you have pandas, inlcude this line below. Not needed if all columns display.
#pd.set_option('display.max_columns', None)

filename_1 = "path/to/file"

filename_2 = "path/to/file"

catalogOne = Table.read(filename_1)
catalogTwo = Table.read(filename_2)

In [ ]:
print(f"The number of columns in `catalogOne` are {len(catalogOne.colnames)}")
print(f"The number of rows are {len(catalogOne)}.")
catalogOne[0:5]

In [ ]:
print(f"The number of columns in `catalogTwo` are {len(catalogTwo.colnames)}")
print(f"The number of rows are {len(catalogTwo)}.")
catalogTwo[0:5]

We start by cleaning our catalogs from unwanted rows/values. 

i.e) Previously we were interested in,

* Observations that are Detected == True. 

* Removing observations that do not have `r2500_temperatures`

In [ ]:
#catalogOne[catalogOne['Detected'] == True]
catalogOne_detected = catalogOne[~np.isnan(catalogOne["r2500_temperature"]) & catalogOne['Detected'] == True]

print(f"The number of rows are {len(catalogOne_detected)}.")
catalogOne_detected[0:3]

Here we join the two tables `catalogOne_detected` and `catalogTwo` from left to right using a shared column. 

i.e) [a, b, c] + [a, d, e, f] --> [a, b, c, d, e, f]

Note that both catalogs need to share the same column name for this method of joining. 
Another method can be used to do a similar join using `np.isin` which I use to double check if the first method is correct. 

In [ ]:
catalog_join = join(catalogOne_detected, catalogTwo, keys='MEM_MATCH_ID')

print(f"The number of columns for table `catalog_join` are {len(catalog_join.colnames)}.")
catalog_join
print(f"The number of rows are {len(catalog_join)}")

Below I use the `np.isin` method to do a similar join as the one above. In this case, what is produced is not a combination of both catalogs but rather reducing the rows of the first catalog to the number of rows shared in both based on similar column values.

i.e) 
if the first value in `catalogOne['MEM_MATCH_ID']` *IS IN* `catalogTwo['MEM_MATCH_ID']`, then *KEEP* that first row in catalogOne.

or 

if the first value `catalogOne['MEM_MATCH_ID']` *IS NOT IN* `catalogTwo['MEM_MATCH_ID']`, then *REMOVE* that row from catalogOne.

In [ ]:
# As a double check for the above code. We'll use a different method to join the dataframes.

catalog_cross1 = catalogOne_detected[np.isin(catalogOne_detected['MEM_MATCH_ID'], catalogTwo['MEM_MATCH_ID'])]

print(f'Dimensions of catalog_cross1 {len(catalog_cross1.colnames)} and {len(catalog_cross1)}.')

# Do the same but in reverse.
catalog_cross2 = catalogTwo[np.isin(catalogTwo['MEM_MATCH_ID'], catalogOne_detected['MEM_MATCH_ID'])]

print(f'Dimensions of catalog_cross2 {len(catalog_cross2.colnames)} and {len(catalog_cross2)}.')

# Check if results we get here are the same as the previous join method.
if len(catalog_join) == len(catalog_cross1) and len(catalog_join) == len(catalog_cross2):
  print("Yuppy!")



Now we're ready to start making our Tx_Tx table with columns we want. We'll be using the `catalog_join` table since it contains the columns from both catalogs and will be easier/cleaner to reference each column. 

Below I've left an example of columns used previously.

General format is,

`NewCatalogName['NewColumnName'] = JoinedCatalogName['NameOfColumn']`

In [ ]:
# Create empty dataframe.
catalog_TxOne_TxTwo = Table()

# Helpful to include obs name, id, and redshifts from both catalogs.
catalog_TxOne_TxTwo["Name"] = catalog_join["Name"] 
catalog_TxOne_TxTwo["MEM_MATCH_ID"] = catalog_join["MEM_MATCH_ID"]
catalog_TxOne_TxTwo["Redshift_matcha"] = catalog_join["Redshift"]
catalog_TxOne_TxTwo["Redshift_xmm"] = catalog_join["Z"]

# Richness
catalog_TxOne_TxTwo["Lambda_matcha"] = catalog_join["lambda"]
catalog_TxOne_TxTwo["Lambda_err_low_matcha"] = catalog_join["lambda_err_low"]
catalog_TxOne_TxTwo["Lambda_err_high_matcha"] = catalog_join["lambda_err_high"]
catalog_TxOne_TxTwo["Lambda_xmm"] = catalog_join["LAMBDA_CHISQ"]
catalog_TxOne_TxTwo["Lambda_err_xmm"] = catalog_join["LAMBDA_CHISQ_E"]

# Below is the temperatures from catalogOne
catalog_TxOne_TxTwo["Tx_r500_matcha"] = catalog_join["r500_temperature"]
catalog_TxOne_TxTwo["Tx_r500_err_high_matcha"] = catalog_join["r500_temperature_err_high"]
catalog_TxOne_TxTwo["Tx_r500_err_low_matcha"] = catalog_join["r500_temperature_err_low"]

# Below is the r500 temperatures from catalogOne
##catalog_TxOne_TxTwo["Tx_r500_err_high_matcha"] = catalog_join["r500_temperature_err_high"]
#catalog_TxOne_TxTwo["Tx_r500_err_low_matcha"] = catalog_join["r500_temperature_err_low"]

# Below is the Tx from catalogTwo
catalog_TxOne_TxTwo["Tx_r500_xmm"] = catalog_join["kT"]
catalog_TxOne_TxTwo["Tx_r500_err_high_xmm"] = catalog_join["kT_p"]
catalog_TxOne_TxTwo["Tx_r500_err_low_xmm"] = catalog_join["kT_m"]

# Below is the SNR columns from both catalogs
# catalog_TxOne_TxTwo["r2500_SNR_matcha"] = catalog_join["r2500_SNR"]
catalog_TxOne_TxTwo["r500_SNR_matcha"] = catalog_join["r500_SNR"]
catalog_TxOne_TxTwo["r500_PNSNR_alt_xmm"] = catalog_join["pnsnr_alt"]

# Show table with dimensions.
print(f"The dimensions of catalog_TxOne_TxTwo are {len(catalog_TxOne_TxTwo.colnames)} and {len(catalog_TxOne_TxTwo)}.")

# Display entire catalog.
catalog_TxOne_TxTwo

Great! Now we have our new catalog. Let's save it!
Make sure to replace `filename.fits` below with a name for your new catalog.

In [ ]:
catalog_TxOne_TxTwo.write("path/to/desired/directory/filename.fits", format="fits")

Done!